In [6]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [8]:
import pandas as pd
df = pd.read_csv("lead.csv")
df.sample(3)

,First Name,Last Name,Email,Company,Job Title,Industry,Company Size
18,Jeb,Bush,jeb@bush-consulting.com,Bush Consulting,Independent Consultant,Politics,1
10,Bruce,Wayne,bruce@waynetech.com,Wayne Enterprises,Chairman,Conglomerate,10000+
16,Tanvir,Ahmed,tanvir@pathao.com,Pathao,Data Scientist,Logistics,500-1000


In [11]:
from typing import TypedDict, Optional

class AgentState(TypedDict):
    # --- 1. INPUT DATA ---
    lead_data: dict  # Raw data from CSV (Name, Email, Company, Job Title, etc.)
    
    # --- 2. QUALIFICATION (From Scorer Agent) ---
    priority: str          # "High", "Medium", "Low"
    priority_score: int    # 1 to 10 (Useful for sorting the final CSV)
    priority_reason: str   # The "Why" behind the score
    
    # --- 3. ENRICHMENT (From Persona Agent) ---
    persona: str           # e.g., "The Data-Driven Executive"
    persona_description: str # Brief profile of their pain points
    
    # --- 4. OUTREACH (From Drafter Agent) ---
    email_subject: str
    email_body: str        # The personalized HTML/Text body
    
    # --- 5. SIMULATION (From Response Agents) ---
    simulated_reply: Optional[str]   # What the AI thinks they would say back
    response_category: Optional[str] # "Interested", "Not Interested", "Auto-reply"
    
    # --- 6. EXECUTION STATUS ---
    status: str            # "Sent", "Failed", or "Pending"

In [21]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq # Switch later
from prompts import LEAD_SCORER_SYSTEM_PROMPT
from schema import LeadScore

load_dotenv()

def get_scorer_agent():
    # Model Setup
    # llm = ChatOpenAI(model="gpt-4o-mini", temperature=0) 
    # To switch to Groq later:
    llm = ChatGroq(model="qwen/qwen3-32b", temperature=0)
    
    # Bind the structured output schema to the LLM
    structured_llm = llm.with_structured_output(LeadScore)
    
    return structured_llm

# --- THE NODE FUNCTION FOR LANGGRAPH ---


In [22]:
def lead_scorer_node(state: AgentState):
    lead_data = state["lead_data"]
    
    # Format the user message with Lead details
    user_message = f"Lead Data: {lead_data}"
    
    # Invoke the agent
    scorer_agent = get_scorer_agent()
    result = scorer_agent.invoke([
        ("system", LEAD_SCORER_SYSTEM_PROMPT),
        ("human", user_message)
    ])
    
    # Update the LangGraph State
    return {
        "priority": result.priority,
        "priority_score": result.score, # Added to state for more detail
        "priority_reason": result.reasoning
    }

## **Test the lead-scorer agent**

In [23]:
# Create a dummy row from your DF
sample_lead = df.iloc[0].to_dict() # Elon Musk

# Create initial state
test_state = {
    "lead_data": sample_lead,
    "priority": "",
    "priority_score": 0,
    "priority_reason": ""
}

# Run node
updated_state = lead_scorer_node(test_state)
print(updated_state)

{'priority': 'High', 'priority_score': 10, 'priority_reason': 'The lead is a CEO of a major Aerospace company with over 10,000 employees. High decision-making authority, industry relevance to AI/automation, and significant budget potential.'}
